In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import plotly.graph_objects as go


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-shows/netflix_titles.csv


# Netflix Exploratory Data Analysis****

The Goal of this analysis is to uncover any insights or trends in the catalogue of films and TV shows available on the Netflix website.

First I will import the dataset that to see the parameters I have to work with.

In [2]:
df = pd.read_csv("/kaggle/input/netflix-shows/netflix_titles.csv")

In [3]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [4]:
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

Now we have the data imported, the above line of code shows the columns that we have to work with many of which are self explanatory, but I will highlight a the below:
* Show ID - This tells us what season the show is, if a TV show 
* Type - Whether it's a Movie or TV-Show
* Listed in - The category that the TV-Show  or film falls under e.g. Documentary or Action
* Rating - the audiences that the film or TV-Show is safe for vieiwing for 

Now that the data is imported, the next step is to Process or clean the data for analysis, I will begin first by checking for any missing values.

In [5]:
missing_data = df.isnull()

In [6]:
missing_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,False,False,False,False,True,False,False,False,False,False,False,False
1,False,False,False,True,False,False,False,False,False,False,False,False
2,False,False,False,False,False,True,False,False,False,False,False,False
3,False,False,False,True,True,True,False,False,False,False,False,False
4,False,False,False,True,False,False,False,False,False,False,False,False


Above I have created a dataframe that tells me for which values are missing, values that are missing are True and those which are not are False. This can be difficult to view so to make it easier to see, let's count the amount of missing data in each column.

In [7]:
for column in missing_data.columns.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print("")

show_id
False    8807
Name: show_id, dtype: int64

type
False    8807
Name: type, dtype: int64

title
False    8807
Name: title, dtype: int64

director
False    6173
True     2634
Name: director, dtype: int64

cast
False    7982
True      825
Name: cast, dtype: int64

country
False    7976
True      831
Name: country, dtype: int64

date_added
False    8797
True       10
Name: date_added, dtype: int64

release_year
False    8807
Name: release_year, dtype: int64

rating
False    8803
True        4
Name: rating, dtype: int64

duration
False    8804
True        3
Name: duration, dtype: int64

listed_in
False    8807
Name: listed_in, dtype: int64

description
False    8807
Name: description, dtype: int64



Now I can see that there a missing values in the columns; 
* Director
* Cast
* Country
* Date-added
* Rating
* Duration

For some columns I will ignore these missing values, e.g cast and director. But I will fix some of the other columns.
First Ratings:

In [8]:
df['rating'].value_counts().sort_values(ascending = False)

TV-MA       3207
TV-14       2160
TV-PG        863
R            799
PG-13        490
TV-Y7        334
TV-Y         307
PG           287
TV-G         220
NR            80
G             41
TV-Y7-FV       6
NC-17          3
UR             3
74 min         1
84 min         1
66 min         1
Name: rating, dtype: int64

We can see here that some of the ratings have durations instead, this may be an error where the data has been transposed

In [9]:
df['duration'].value_counts().sort_values(ascending=False)


1 Season     1793
2 Seasons     425
3 Seasons     199
90 min        152
94 min        146
             ... 
167 min         1
43 min          1
196 min         1
200 min         1
191 min         1
Name: duration, Length: 220, dtype: int64

Let's isolate these rows and see what has happened.

In [10]:
print(df.loc[df['rating'] == '66 min'])
print(df.loc[df['rating'] == '84 min'])
print(df.loc[df['rating'] == '74 min'])



     show_id   type                                 title    director  \
5813   s5814  Movie  Louis C.K.: Live at the Comedy Store  Louis C.K.   

            cast        country       date_added  release_year  rating  \
5813  Louis C.K.  United States  August 15, 2016          2015  66 min   

     duration listed_in                                        description  
5813      NaN    Movies  The comic puts his trademark hilarious/thought...  
     show_id   type                  title    director        cast  \
5794   s5795  Movie  Louis C.K.: Hilarious  Louis C.K.  Louis C.K.   

            country          date_added  release_year  rating duration  \
5794  United States  September 16, 2016          2010  84 min      NaN   

     listed_in                                        description  
5794    Movies  Emmy-winning comedy writer Louis C.K. brings h...  
     show_id   type            title    director        cast        country  \
5541   s5542  Movie  Louis C.K. 2017  Louis C

There seems to be an error with the titles by the comedian Louis CK, a quick look up on the Netflix website will tell us what the correct rating is for these shows.

In [11]:
#Louis C.K.: Live at the Comedy Store 1 hr 5 min (15 - tvma)
#Louis C.K.: Hilarious 1 hr 23 min (18)
#Louis C.K. 2017 1 hr 14 min (15 - tvma)


Now that we have the correct rating we can now move the values for duration into the correct column and input the correct rating also

In [12]:
df.iloc[5813, df.columns.get_loc('duration')] = "66 min"
df.iloc[5813, df.columns.get_loc('rating')] = "TV-MA"
df.iloc[5794, df.columns.get_loc('duration')] = "84 min"
df.iloc[5794, df.columns.get_loc('rating')] = "TV-MA"
df.iloc[5541, df.columns.get_loc('duration')] = "74 min"
df.iloc[5541, df.columns.get_loc('rating')] = "TV-MA"

In [13]:
df.iloc[5813]
df.iloc[5794]
df.iloc[5541]

show_id                                                     s5542
type                                                        Movie
title                                             Louis C.K. 2017
director                                               Louis C.K.
cast                                                   Louis C.K.
country                                             United States
date_added                                          April 4, 2017
release_year                                                 2017
rating                                                      TV-MA
duration                                                   74 min
listed_in                                                  Movies
description     Louis C.K. muses on religion, eternal love, gi...
Name: 5541, dtype: object

In [14]:
print(df['duration'].isnull().sum())



0


We have dealt with mising values in the duration column, now let's turn our attention back to ratings column, rather than searching the titles of the missing values, instead I am going to replace the misisng values with the most common; TV-MA

In [15]:
df["rating"].replace(np.nan, "TV-MA", inplace=True)

In [16]:
df['rating'].isnull().sum()

0

Great, another column down. Now let's take a look at Date_added.
The data in this column shows the date that the title was added to Netflix, this data may be hard to find, instead of wasting time, I'm going to remove these rows from the analysis.

In [17]:
df.dropna(subset=["date_added"], axis=0, inplace=True)
# reset index, because we droped some rows
df.reset_index(drop=True, inplace=True)

In [18]:
df['date_added'].isnull().sum()

0

Now we see that we also have no null rows in the Date Added column, great. For the columns Director, Cast and Country that still contain missing data, I will cgoose to ignore these. Now that I am happy with the state of the data I can begin my analsysis.

The aim of this project is to explore the data that is available on Netflix, from the column names we know that Netlix offers TV-Shows and Movies. But in what proportion?

In [19]:
pie_data = df.groupby(['type'])['show_id'].count().reset_index()
pie_data

,type,show_id
0,Movie,6131
1,TV Show,2666


In [20]:
fig = px.pie(pie_data, values='show_id', names='type', title='Proportion of Movies vs TV Shows on Netflix') 
fig.show()

This pie chart shows us that the majority of the Netflix catalogue consists of movies, about a 70:30 split between movies and TV Shows. Given that the majority of the titles on Netflix, let's take a closer look at them.

In [21]:
hist_data = df.groupby(['release_year','type'])['show_id'].count().reset_index()
hist_data.head()

,release_year,type,show_id
0,1925,TV Show,1
1,1942,Movie,2
2,1943,Movie,3
3,1944,Movie,3
4,1945,Movie,3


In [22]:
fig = px.bar(hist_data, x="release_year", y='show_id', color="type")
fig.show()

This is a bar chart to show the the amount of movies and TV-shows on Netflix by release year. On the x-axis we have release year and on the y-axis the sum of titles released, with Movies indicated by the red bar and TV-shows indicated by thr blue .
From the histogram, we can see that the Netflix catalogue has a range of movies from 1942, to 2021 with the majority of the movies being released in the year 2021 

No we have an idea of the quantity of movies and there release dates I'd like to see if there is any information that can be 
gathered from the country data.
I begin by creating a new dataframe of just the countries 

In [23]:
countries = df['country'].value_counts()

countries.head(10)

United States     2812
India              972
United Kingdom     418
Japan              244
South Korea        199
Canada             181
Spain              145
France             124
Mexico             110
Egypt              106
Name: country, dtype: int64

Now I want to create a list of 

In [24]:
country_list = ['United States', 'India', 'United Kingdom', 'Japan', 'South Korea', 'Canada',
              'Spain', 'France', 'Mexico', 'Egypt']

I want to drop the countries that aren't in the top 10 country list and rename the show ID column to count or sum.

In [25]:
country_df = df[df['country'].isin(country_list)].reset_index()
country_df =country_df.groupby(['country','type'])['show_id'].count().reset_index()
country_df.head()

,country,type,show_id
0,Canada,Movie,122
1,Canada,TV Show,59
2,Egypt,Movie,92
3,Egypt,TV Show,14
4,France,Movie,75


In [26]:
fig = px.bar(country_df, x="country", y="show_id",
             color='type', barmode='group')
fig.show()

This graph shows the top 10 content makers for Netflix and that the United States produces the most content on Netflix 
For TV shows this is followed secondly by India, then the UK
In 10th place for movies we have Egypt

In [27]:
df.dtypes

show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object

In [28]:
df['year_added'] = str(df['date_added'])

In [29]:
df.dtypes

show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
year_added      object
dtype: object

In [30]:
df['date_added'] = pd.to_datetime(df['date_added'])



In [31]:
df.dtypes

show_id                 object
type                    object
title                   object
director                object
cast                    object
country                 object
date_added      datetime64[ns]
release_year             int64
rating                  object
duration                object
listed_in               object
description             object
year_added              object
dtype: object

In [32]:
import datetime

In [33]:
df['time'] = pd.DatetimeIndex(df['date_added']).year


In [34]:
df['time'].head(10)

0    2021
1    2021
2    2021
3    2021
4    2021
5    2021
6    2021
7    2021
8    2021
9    2021
Name: time, dtype: int64

In [35]:
df['long'] = df['time']-df['release_year']

In [36]:
df['long']

0        1
1        0
2        0
3        0
4        0
        ..
8792    12
8793     1
8794    10
8795    14
8796     4
Name: long, Length: 8797, dtype: int64

In [37]:
df.drop(df[df.long < 0].index, inplace=True)

In [38]:
df['long'].describe()

count    8783.000000
mean        4.697825
std         8.790808
min         0.000000
25%         0.000000
50%         1.000000
75%         5.000000
max        93.000000
Name: long, dtype: float64

In [39]:
time_to_screen = df.groupby(['long'])['show_id'].count().reset_index()

fig = px.bar(time_to_screen, x='long', y = 'show_id')
fig.show()

Here we can see that the majority of Netflix shows and movies are relased and are on Netflix within one year, this is likely due to the productions released by Netflix Studios 

Ensuring a regurlar catalogue of new shows entices the viewers to keep coming back to Netflix and keeps retention 